<a href="https://colab.research.google.com/github/sxyang1206/IBM-University/blob/main/uni_text_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
pip install rapidfuzz datasketch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from wordcloud import WordCloud
from concurrent.futures import ProcessPoolExecutor

In [ ]:
df_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/University_1to124_ratings.csv')
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12717 entries, 0 to 12716
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   University  12717 non-null  object
 1   Review      12717 non-null  object
dtypes: object(2)
memory usage: 198.8+ KB


In [ ]:
def remove_name(text):
    parts = str(text).split('\n', 1)
    return parts[1] if len(parts) > 1 else parts[0]

In [ ]:
from datasketch import MinHash, MinHashLSH
from nltk.tokenize import word_tokenize

lsh = MinHashLSH(threshold=0.8, num_perm=128)

minhashes = {}
for i, text in enumerate(df_review['Review']):
    tokens = word_tokenize(text.lower())
    m = MinHash(num_perm=128)
    for token in tokens:
        m.update(token.encode('utf8'))
    lsh.insert(f"text-{i}", m)
    minhashes[f"text-{i}"] = m

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from rapidfuzz import fuzz

def check_duplicates(text_id):
    m = minhashes[text_id]
    similar_texts = lsh.query(m)
    duplicates = []
    for sim_text_id in similar_texts:
        if sim_text_id != text_id:
            # 使用RapidFuzz计算具体的相似度
            original_text_index = int(text_id.split('-')[1])
            similar_text_index = int(sim_text_id.split('-')[1])
            original_text = df_review['Review'].iloc[original_text_index]
            similar_text = df_review['Review'].iloc[similar_text_index]
            score = fuzz.ratio(original_text, similar_text)
            if score > 80:  # 设定相似度阈值
                duplicates.append(sim_text_id)
    return text_id, duplicates

duplicates = set()
with ProcessPoolExecutor() as executor:
    results = executor.map(check_duplicates, minhashes.keys())
    for text_id, sim_duplicates in results:
        duplicates.update(sim_duplicates)

In [ ]:
indices_to_remove = [int(text_id.split('-')[1]) for text_id in duplicates]
df_review = df_review.drop(index=indices_to_remove).reset_index(drop=True)

In [ ]:
df_review

,University,Review
0,The University of Leeds,"Anonymous\nLeeds university is great, from the..."
1,The University of Leeds,Anonymous\nThe union is the most welcoming tha...
2,The University of Leeds,Anonymous\nExcellent coaching with professiona...
3,The University of Leeds,"Anonymous\nWicked uni, nice campus with lots o..."
4,The University of Leeds,Anonymous\nI thoroughly enjoyed my 3 years at ...
...,...,...
12462,University of Central Lancashire,Anonymous\nIt's good. The facilities have rece...
12463,University of Central Lancashire,Anonymous\nGreat place lots of accommodation f...
12464,University of Central Lancashire,Anonymous\nThe tutors teach to a very high sta...
12465,University of Central Lancashire,"Anonymous\nLove my small class, I get more out..."


In [ ]:
  def clean_comment(text):
      text = re.sub(r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b|\b\d{1,2}:\d{2}(?::\d{2})?\b|<.*?>|http\S+', '', text)
      text = re.sub('[^A-Za-z0-9\s]+', '', text).lower()
      words = text.split()

      text = [word.lower().translate(str.maketrans('', '', string.punctuation)) for word in words]
      stop_words = stopwords.words('english')
      text = [word for word in words if word not in stop_words]

      lemmatizer = WordNetLemmatizer()
      text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]
      text = [word for word in text if word not in stop_words]

      processed_text = ' '.join(text)

      blob = TextBlob(processed_text)

      processed_text = blob.correct()
      text = str(processed_text).split()

      text = [word for word in text if word not in stop_words]
      text = ' '.join(text)

      return str(text)

In [ ]:
df_review.to_csv('/content/drive/MyDrive/Colab Notebooks//University_1to124_ratings_clean_full.csv', index=False)


In [ ]:
df_review = df_review.drop_duplicates(subset=['Review'],keep=False)
df_review['Review'] = df_review['Review'].apply(lambda x: remove_name(x))
df_review['Word_Count'] = df_review['Review'].apply(lambda x: len(x.split()))
df_review = df_review[df_review['Word_Count'] >= 5]

In [ ]:
def parallel_clean_comments(df):
    with ProcessPoolExecutor() as executor:
        cleaned_texts = list(executor.map(clean_comment, df_review['Review']))
    return cleaned_texts

df_review['Review'] = parallel_clean_comments(df_review)

df_review['Word_Count'] = df_review['Review'].apply(lambda x: len(x.split()))
df_review.describe()

,Word_Count
count,12445.000000
mean,28.197670
std,21.117593
min,3.000000
25%,15.000000
50%,22.000000
75%,34.000000
max,322.000000


In [ ]:
df_review = df_review[df_review['Word_Count'] >= 5]
df_review.to_csv('/content/drive/MyDrive/Colab Notebooks//University_1to124_ratings_clean_full.csv', index=False)
df_review.describe()

,Word_Count
count,12440.000000
mean,28.207556
std,21.116074
min,5.000000
25%,15.000000
50%,22.000000
75%,34.000000
max,322.000000


# BERT

In [ ]:
df_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/University_1to124_ratings_clean_full.csv')


In [ ]:
df_review

,University,Review,Word_Count
0,The University of Leeds,leeds university great outstanding staff amazi...,57
1,The University of Leeds,union welcoming ever come across university re...,17
2,The University of Leeds,excellent reaching professional lab practical ...,11
3,The University of Leeds,wicked un nice camps lot social area lot libra...,15
4,The University of Leeds,thoroughly enjoyed 3 year leeds university ama...,71
...,...,...,...
12435,University of Central Lancashire,good facility recently updated lot support opp...,18
12436,University of Central Lancashire,great place lot accommodation student un bar c...,24
12437,University of Central Lancashire,tutor teach high standard involved student fac...,17
12438,University of Central Lancashire,love small class get output 11 time help need ...,20


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from scipy.special import softmax
import pandas as pd
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
def sentiment_analysis(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores

labels = ['NEGATIVE', 'NEUTRAL', 'POSITIVE']

results = df_review['Review'].apply(sentiment_analysis)

for i, label in enumerate(labels):
    df_review[label] = results.apply(lambda x: np.round(float(x[i]), 4))


df_review.head()

,University,Review,Word_Count,NEGATIVE,NEUTRAL,POSITIVE
0,The University of Leeds,leeds university great outstanding staff amazi...,57,0.0030,0.0200,0.9770
1,The University of Leeds,union welcoming ever come across university re...,17,0.0131,0.3507,0.6362
2,The University of Leeds,excellent reaching professional lab practical ...,11,0.0027,0.0706,0.9266
3,The University of Leeds,wicked un nice camps lot social area lot libra...,15,0.0039,0.0253,0.9709
4,The University of Leeds,thoroughly enjoyed 3 year leeds university ama...,71,0.0025,0.0280,0.9696


In [ ]:
df_review.describe()

,Word_Count,NEGATIVE,NEUTRAL,POSITIVE
count,12440.000000,12440.000000,12440.000000,12440.000000
mean,28.207556,0.047818,0.151503,0.800678
std,21.116074,0.136795,0.171679,0.254183
min,5.000000,0.001100,0.007100,0.001900
25%,15.000000,0.003200,0.034700,0.761175
50%,22.000000,0.005550,0.075500,0.917550
75%,34.000000,0.016825,0.204550,0.961600
max,322.000000,0.980200,0.937200,0.991400


In [ ]:
df_review

,University,Review,Word_Count,NEGATIVE,NEUTRAL,POSITIVE
0,The University of Leeds,leeds university great outstanding staff amazi...,57,0.0030,0.0200,0.9770
1,The University of Leeds,union welcoming ever come across university re...,19,0.0111,0.3509,0.6380
2,The University of Leeds,excellent reaching professional lab practical ...,11,0.0027,0.0706,0.9266
3,The University of Leeds,wicked un nice camps lot social area lot libra...,15,0.0039,0.0253,0.9709
4,The University of Leeds,thoroughly enjoyed 3 year leeds university ama...,71,0.0025,0.0280,0.9696
...,...,...,...,...,...,...
12435,University of Central Lancashire,good facility recently updated lot support opp...,18,0.0048,0.0681,0.9272
12436,University of Central Lancashire,great place lot accommodation student un bar c...,24,0.0041,0.0505,0.9454
12437,University of Central Lancashire,tutor teach high standard involved student fac...,17,0.0028,0.0263,0.9709
12438,University of Central Lancashire,love small class get output 11 time help need ...,20,0.0056,0.1154,0.8790


In [ ]:
labels = ['NEGATIVE', 'NEUTRAL', 'POSITIVE']  # 根据模型输出调整


def dominant_emotion(row):
    emotions = {label: row[label] for label in labels}
    dominant = max(emotions, key=emotions.get)
    return dominant, emotions[dominant]

df_review['Dominant_Emotion'], df_review['Dominant_Score'] = zip(*df_review.apply(dominant_emotion, axis=1))


In [ ]:
df_review.to_csv('/content/drive/MyDrive/Colab Notebooks//University_1to124_ratings_senti_full.csv', index=False)

In [ ]:
df_review.to_csv('/content/drive/MyDrive/Colab Notebooks//University_1to124_ratings_senti_full.csv', index=False)
df_review = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/University_1to124_ratings_senti_full.csv')

# LDA

In [ ]:
import gensim
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora

In [ ]:
def extract_topics(lda_model, num_topics):
    topic_data = []
    for i in range(num_topics):
        words = lda_model.show_topic(i, topn=10)
        for word, prob in words:
            topic_data.append([i, word, round(prob, 4)])
    return pd.DataFrame(topic_data, columns=['Topic', 'Keyword', 'Score'])

In [ ]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

df_review['Tokenized_Review'] = df_review['Review'].apply(preprocess_text)

dictionary = corpora.Dictionary(df_review['Tokenized_Review'])
dictionary.filter_extremes(no_below=8, no_above=0.8)
corpus = [dictionary.doc2bow(doc) for doc in df_review['Tokenized_Review']]

coherence_values_cv = []
coherence_values_umass = []
coherence_values_c_npmi = []
model_list = []

for num_topics in range(2, 15):
    lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=80)
    model_list.append(lda_model)

    coherence_model_cv = CoherenceModel(model=lda_model, texts=df_review['Tokenized_Review'], dictionary=dictionary, coherence='c_v')
    coherence_value_cv = coherence_model_cv.get_coherence()
    coherence_values_cv.append(coherence_value_cv)

    coherence_model_umass = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, coherence='u_mass')
    coherence_value_umass = coherence_model_umass.get_coherence()
    coherence_values_umass.append(coherence_value_umass)

    coherence_model_c_npmi = CoherenceModel(model=lda_model, texts=df_review['Tokenized_Review'], dictionary=dictionary, coherence='c_npmi')
    coherence_value_c_npmi = coherence_model_c_npmi.get_coherence()
    coherence_values_c_npmi.append(coherence_value_c_npmi)

    print(f"Num Topics: {num_topics}, c_v Coherence Value: {coherence_value_cv}, u_mass Coherence Value: {coherence_value_umass}， c_npmi Coherence Value: {coherence_value_c_npmi}")

best_num_topics_cv = coherence_values_cv.index(max(coherence_values_cv)) + 2
print(f"Best Number of Topics based on c_v: {best_num_topics_cv}")

best_num_topics_umass = coherence_values_umass.index(max(coherence_values_umass)) + 2
print(f"Best Number of Topics based on u_mass: {best_num_topics_umass}")

best_num_topics_c_npmi = coherence_values_c_npmi.index(max(coherence_values_c_npmi)) + 2
print(f"Best Number of Topics based on c_npmi: {best_num_topics_c_npmi}")

Num Topics: 2, c_v Coherence Value: 0.41846124004638385, u_mass Coherence Value: -1.7377375727425215， c_npmi Coherence Value: -0.00015257431271034497
Num Topics: 3, c_v Coherence Value: 0.4331206774723348, u_mass Coherence Value: -1.8186356073958105， c_npmi Coherence Value: 0.001362943093707186
Num Topics: 4, c_v Coherence Value: 0.4492862031471905, u_mass Coherence Value: -1.8953878340676973， c_npmi Coherence Value: 0.007433674625693198
Num Topics: 5, c_v Coherence Value: 0.4653968268632623, u_mass Coherence Value: -1.875931717293374， c_npmi Coherence Value: 0.013110335299496758
Num Topics: 6, c_v Coherence Value: 0.4531280520000412, u_mass Coherence Value: -1.8940317011682508， c_npmi Coherence Value: 0.010718224801305633
Num Topics: 7, c_v Coherence Value: 0.4689949204133147, u_mass Coherence Value: -2.0018360980368213， c_npmi Coherence Value: 0.014211358973022671
Num Topics: 8, c_v Coherence Value: 0.47670333851735486, u_mass Coherence Value: -2.104488228098446， c_npmi Coherence Val

In [ ]:
dictionary = corpora.Dictionary(df_review['Tokenized_Review'])
dictionary.filter_extremes(no_below=5, no_above=0.8)
corpus = [dictionary.doc2bow(doc) for doc in df_review['Tokenized_Review']]
lda_model = LdaModel(corpus, num_topics=8, id2word=dictionary, passes=200)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)


doc_topics = [lda_model.get_document_topics(bow) for bow in corpus]
def get_dominant_topic(doc_topics):
    return max(doc_topics, key=lambda x: x[1])[0]

df_review['Topic'] = [get_dominant_topic(doc) for doc in doc_topics]
df_topics = extract_topics(lda_model, 5)


(0, '0.061*"university" + 0.026*"student" + 0.025*"best" + 0.019*"would" + 0.019*"great" + 0.018*"one" + 0.016*"life" + 0.016*"recommend" + 0.015*"people" + 0.013*"amazing"')
(1, '0.038*"good" + 0.026*"however" + 0.020*"course" + 0.015*"student" + 0.015*"could" + 0.014*"camps" + 0.013*"un" + 0.012*"quite" + 0.011*"time" + 0.011*"overall"')
(2, '0.069*"good" + 0.062*"great" + 0.045*"facility" + 0.043*"camps" + 0.036*"student" + 0.028*"university" + 0.022*"staff" + 0.021*"union" + 0.021*"really" + 0.020*"course"')
(3, '0.069*"student" + 0.042*"career" + 0.037*"service" + 0.024*"union" + 0.022*"university" + 0.019*"many" + 0.018*"opportunity" + 0.018*"event" + 0.017*"help" + 0.015*"support"')
(4, '0.046*"course" + 0.017*"facility" + 0.017*"lecture" + 0.016*"lecturer" + 0.014*"university" + 0.014*"well" + 0.012*"study" + 0.011*"student" + 0.011*"module" + 0.010*"year"')
(5, '0.039*"university" + 0.029*"really" + 0.028*"help" + 0.019*"year" + 0.019*"always" + 0.016*"amazing" + 0.015*"feel" 

In [ ]:
dictionary = corpora.Dictionary(df_review['Tokenized_Review'])
dictionary.filter_extremes(no_below=5, no_above=0.8)
corpus = [dictionary.doc2bow(doc) for doc in df_review['Tokenized_Review']]
lda_model = LdaModel(corpus, num_topics=8, id2word=dictionary, passes=300)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)



(0, '0.056*"student" + 0.052*"university" + 0.026*"great" + 0.022*"society" + 0.017*"many" + 0.013*"union" + 0.012*"sport" + 0.011*"people" + 0.011*"camps" + 0.011*"opportunity"')
(1, '0.049*"course" + 0.044*"year" + 0.028*"university" + 0.017*"experience" + 0.015*"first" + 0.014*"study" + 0.010*"give" + 0.009*"studying" + 0.009*"time" + 0.009*"would"')
(2, '0.098*"good" + 0.046*"facility" + 0.041*"course" + 0.035*"great" + 0.035*"student" + 0.032*"camps" + 0.027*"really" + 0.023*"well" + 0.021*"union" + 0.020*"lecturer"')
(3, '0.076*"camps" + 0.057*"city" + 0.032*"great" + 0.025*"close" + 0.025*"location" + 0.021*"centre" + 0.019*"everything" + 0.018*"town" + 0.017*"university" + 0.016*"easy"')
(4, '0.059*"camps" + 0.037*"library" + 0.026*"building" + 0.023*"facility" + 0.019*"good" + 0.018*"study" + 0.018*"space" + 0.018*"food" + 0.017*"student" + 0.017*"lot"')
(5, '0.032*"un" + 0.026*"university" + 0.023*"really" + 0.020*"like" + 0.018*"good" + 0.017*"great" + 0.016*"would" + 0.014*

In [ ]:
dictionary = corpora.Dictionary(df_review['Tokenized_Review'])
dictionary.filter_extremes(no_below=5, no_above=0.8)
corpus = [dictionary.doc2bow(doc) for doc in df_review['Tokenized_Review']]
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=200)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)



(0, '0.032*"camps" + 0.026*"good" + 0.021*"library" + 0.019*"student" + 0.018*"building" + 0.018*"facility" + 0.015*"course" + 0.014*"lecture" + 0.014*"however" + 0.010*"food"')
(1, '0.021*"really" + 0.018*"like" + 0.017*"un" + 0.017*"course" + 0.015*"year" + 0.015*"feel" + 0.014*"university" + 0.013*"get" + 0.012*"time" + 0.010*"people"')
(2, '0.036*"good" + 0.032*"student" + 0.030*"great" + 0.029*"help" + 0.028*"camps" + 0.026*"facility" + 0.026*"helpful" + 0.024*"university" + 0.023*"staff" + 0.022*"course"')
(3, '0.053*"university" + 0.027*"student" + 0.020*"course" + 0.017*"year" + 0.017*"experience" + 0.013*"great" + 0.012*"best" + 0.010*"support" + 0.010*"teaching" + 0.010*"opportunity"')
(4, '0.047*"camps" + 0.038*"great" + 0.029*"university" + 0.028*"student" + 0.021*"city" + 0.019*"facility" + 0.018*"good" + 0.017*"sport" + 0.013*"union" + 0.012*"place"')


In [ ]:
df_topics.to_csv('/content/drive/MyDrive/Colab Notebooks/uni_review_topics.csv', index=False)